# Image generation

In Part 1 of this course, we focused mainly on models that were useful for classification. However, many applications require generating much higher dimensional results, such as images and sentences. Examples include:
* Text: neural translation, text to speech, image captioning
* Image: Segmentation, artistic filters, image sharpening and cleaning

In [ ]:
%matplotlib inline
import importlib
import numpy as np
import glob
import random
from PIL import Image

from scipy.misc import imsave
from keras import metrics

from vgg16_avg import VGG16_Avg

In [ ]:
import keras.backend as K
def limit_mem():
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.Session(config=cfg))
limit_mem()

In [ ]:
DEFAULT_ITERATIONS=500

## Neural style transfer

The first use case of an image to image architecture we're going to look at is neural style transfer, using the approach in [this paper](https://arxiv.org/abs/1508.06576). This is a fairly popular application of deep learning in which an image is recreated in the style of a work of art, such as Van Gogh's Starry Night. For more information about the use of neural networks in art, see this [Scientific American article](https://blogs.scientificamerican.com/sa-visual/neural-networks-for-artists/) or [Google's Magenta Project](https://magenta.tensorflow.org/welcome-to-magenta).

### Setup

In [ ]:
datapath = '/home/bfortuner/workplace/deephacks/'
imgpath = datapath+'images/'
stylepath = datapath+'styles/'
resultspath = datapath+'results/'
segmentations = imgpath+'segmentations/'
IMG_EXTEN='.jpg'
STYLE_FILENAME='mariana_s_portrait__avatar_style__by_kalisami-d9mhmho.png' #'starrynight.jpg'
TEST_PHOTO_FILENAME='cat.jpg'#good_dog_img_for_testing.jpg'

In [ ]:
#Save Filenames
import os
import pickle

fnames = list(glob.iglob(imgpath+'*'+IMG_EXTEN))
pickle.dump(fnames, open(imgpath+'fnames.pkl', 'wb'))

img_filenames = []
img_filenames_w_path = []
for (dirpath, dirnames, filenames) in os.walk(imgpath):
    img_filenames.extend(filenames)
    img_filenames_w_path.extend(os.path.join(dirpath, f) 
                                for f in filenames if f.endswith(IMG_EXTEN))

print (img_filenames[:5])
print (img_filenames_w_path[:5])

# Save filenames to file with pickle
filenames_path = imgpath+"fnamesfullpath.p" 
pickle.dump(img_filenames_w_path, open(filenames_path, "wb" ) )

In [ ]:
# Load filenames
fnames = pickle.load(open(filenames_path, 'rb'))
N_files = len(fnames); N_files

### Prepare Random Image

In [ ]:
img = Image.open(fnames[random.randrange(N_files)]); 
img
#img.crop()

# Use Specific Image

In [ ]:
img = Image.open(imgpath+TEST_PHOTO_FILENAME) 
print(img.size)
img 

We need to subtract the mean of each channel of the imagenet data and reverse the order of RGB->BGR since those are the preprocessing steps that the VGG authors did - so their model won't work unless we do the same thing.
We can do this in one step using broadcasting, which is a topic we'll be returning to many times during this course.

In [ ]:
rn_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32)
preproc = lambda x: (x - rn_mean)[:, :, :, ::-1]

In [ ]:
img_arr = preproc(np.expand_dims(np.array(img), 0))
img_shape = img_arr.shape
img_arr.shape

When we generate images from this network, we'll need to undo the above preprocessing in order to view them.

In [ ]:
deproc = lambda x,s: np.clip(x.reshape(s)[:, :, :, ::-1] + rn_mean, 0, 255)

In [ ]:
import matplotlib.pyplot as plt

def plot_arr(arr): 
    plt.imshow(deproc(arr,arr.shape)[0].astype('uint8'))

In [ ]:
plot_arr(img_arr)

### Load Style

In [ ]:
style_img = Image.open(stylepath+STYLE_FILENAME)
print ("original size" + str(style_img.size))

style_img = style_img.resize(np.divide(style_img.size,3).astype('int32'));
print ("new size" + str(style_img.size))
style_img

In [ ]:
style_arr = preproc(np.expand_dims(style_img,0)[:,:,:,:3])

### Resize Images So They Match

In [ ]:
style_w = style_arr.shape[1]
style_h = style_arr.shape[2]
img_w = img_arr.shape[1]
img_h = img_arr.shape[2]

In [ ]:
img_input = img_arr[:,:style_w,:style_h]
style_input = style_arr[:,:img_w,:img_h]
style_shape = style_input.shape
img_shape = img_input.shape

plot_arr(img_input)
print(img_input.shape)

# Our generated image should have the following shape
output_img_shape = style_input.shape

In [ ]:
plot_arr(style_input)
print(style_input.shape)

In [ ]:
print(img_shape)
print(style_shape)

### Recreate Content

The first step in style transfer is understanding how to recreate an image from noise based on "content loss", which is the amount of difference between activations in some layer. In earlier layes, content loss is very similar to per-pixel loss, but in later layers it is capturing the "meaning" of a part of an image, rather than the specific details.

To do this, we first take a CNN and pass an image through it. We then pass a "noise image" (i.e. random pixel values) through the same CNN. At some layer, we compare the outputs from it for both images. We then use a MSE to compare the activations of these two outputs. 

The interesting part is that now, instead of updating the parameters of the CNN, we update the pixels of the noisy image. In other words, our goal is to alter the noisy image so as to minimize the difference between the original image's output at some convolutional layer with the output of the noisy image at the same layer.

In order to construct this architecture, we're going to be working with keras.backend, which is an abstraction layer that allows us to target both theano and tensorflow with the same code.

The CNN we'll use is VGG16, but with a twist. Previously we've always used Vgg with max pooling, and this was useful for image classification. It's not as useful in this case however, because max pooling loses information about the original input area. Instead we will use average pooling, as this does not throw away as much information.

In [ ]:
img_model = VGG16_Avg(include_top=False)

Here we're grabbing the activations from near the end of the convolutional model).

In [ ]:
img_conv_layer = img_model.get_layer('block4_conv1').output
img_conv_layer

And let's calculate the target activations for this layer:

In [ ]:
from keras.models import Model
img_conv_layer_model = Model(img_model.input, img_conv_layer)
img_activations = K.variable(img_conv_layer_model.predict(img_input))
img_activations

In our implementation, we need to define an object that will allow us to separately access the loss function and gradients of a function, since that is what scikit-learn's optimizers require.

In [ ]:
class Evaluator(object):
    def __init__(self, f, shp): self.f, self.shp = f, shp
        
    def loss(self, x):
        loss_, self.grad_values = self.f([x.reshape(self.shp)])
        return loss_.astype(np.float64)

    def grads(self, x): return self.grad_values.flatten().astype(np.float64)

We'll define our loss function to calculate the mean squared error between the two outputs at the specified convolutional layer.

In [ ]:
img_loss = metrics.mse(img_conv_layer, img_activations)
img_grads = K.gradients(img_loss, img_model.input)
fn = K.function([img_model.input], [img_loss]+img_grads)
evaluator = Evaluator(fn, img_shape)

Now we're going to optimize this loss function with a deterministic approach to optimization that uses a line search, which we can implement with sklearn's `fmin_l_bfgs_b` funtionc. 

In [ ]:
from scipy.optimize import fmin_l_bfgs_b
def solve_image(eval_obj, niter, x, shape):
    for i in range(niter):
        x, min_val, info = fmin_l_bfgs_b(eval_obj.loss, x.flatten(),
                                         fprime=eval_obj.grads, maxfun=20)
        x = np.clip(x, -127,127)
        print('Current loss value:', min_val)
        imsave(f'{resultspath}/res_at_iteration_{i}.png', deproc(x.copy(), shape)[0])
    return x

Next we need to generate a random image.

In [ ]:
def gen_rand_noise_img(shape):
    return np.random.uniform(-2.5, 2.5, shape)

rand_noise_img = gen_rand_noise_img(img_shape)
plt.imshow(rand_noise_img[0]);
assert img_shape == rand_noise_img.shape
print (rand_noise_img.shape)

Now we'll run through this optimization approach ten times and train the noise image's pixels as desired.

In [ ]:
iterations=DEFAULT_ITERATIONS

In [ ]:
recreated_img = solve_image(evaluator, iterations, rand_noise_img, img_shape)

Our result by comparing output at conv 1 of last block (5) is fairly amorphous, but still easily recognizable as a bird. Notice that the things it has reconstructed particularly well are those things that we expect Vgg16 to be good at recognizing, such as an eye or a beak.

If instead we optimized by calculating loss from the output of conv 1 of 4th block, our trained image looks much more like the original. This makes sense because with less transformations to go through, comparing at an earlier layer means that we have a smaller receptive field and the features are more based on geometric details rather than broad features.

In [ ]:
Image.open(resultspath+'res_at_iteration_%s.png' % (iterations-1))

In [ ]:
from IPython.display import HTML
from matplotlib import animation, rc

In [ ]:
fig, ax = plt.subplots()
def animate(i):
    ax.imshow(Image.open(f'{resultspath}res_at_iteration_{i}.png'))

The optimizer first focuses on the important details of the bird, before trying to match the background.

In [ ]:
anim = animation.FuncAnimation(fig, animate, frames=DEFAULT_ITERATIONS, interval=200)
HTML(anim.to_html5_video())

# Recreate style

Now that we've learned how to recreate an input image, we'll move onto attempting to recreate *style*. By "style", we mean the color palette and texture of an image. Unlike recreating based on content, with style we are not concerned about the actual structure of what we're creating, all we care about is that it captures this concept of "style".

Here are some examples of images we can extract style from.

We're going to repeat the same approach as before, but with some differences.

### Initialize VGG Model for Style

In [ ]:
style_model = VGG16_Avg(include_top=False, input_shape=style_shape[1:])
style_layer_outputs = {l.name: l.output for l in style_model.layers}

One thing to notice is that we're actually going to be calculating the loss function multiple layers, rather than just one. (Note however that there's no reason you couldn't try using multiple layers in your content loss function, if you wanted to try that).

In [ ]:
style_layers = [style_layer_outputs['block{}_conv1'.format(o)] for o in range(1,4)]

In [ ]:
style_layers_model = Model(style_model.input, style_layers)
style_activations = [K.variable(o) for o in style_layers_model.predict(style_input)]

The key difference is our choice of loss function. Whereas before we were calculating mse of the raw convolutional outputs, here we transform them into the "gramian matrix" of their channels (that is, the product of a matrix and its transpose) before taking their mse. It's unclear why this helps us achieve our goal, but it works. One thought is that the gramian  shows how our features at that convolutional layer correlate, and completely removes all location information. So matching the gram matrix of channels can only match some type of texture information, not location information.

In [ ]:
def gram_matrix(x):
    # We want each row to be a channel, and the columns to be flattened x,y locations
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    # The dot product of this with its transpose shows the correlation 
    # between each pair of channels
    return K.dot(features, K.transpose(features)) / x.get_shape().num_elements()

In [ ]:
def get_style_loss(x, targ): 
    return metrics.mse(gram_matrix(x), gram_matrix(targ))

In [ ]:
style_loss = sum(get_style_loss(l1[0], l2[0]) for l1,l2 in zip(style_layers, style_activations))
style_grads = K.gradients(style_loss, style_model.input)
style_fn = K.function([style_model.input], [style_loss]+style_grads)
style_evaluator = Evaluator(style_fn, style_shape)

We then solve as we did before.

In [ ]:
iterations=DEFAULT_ITERATIONS
rand_noise_img = gen_rand_noise_img(style_shape)

In [ ]:
solve_image(style_evaluator, iterations, rand_noise_img, style_shape)

Our results are stunning. By transforming the convolutional outputs to the gramian, we are somehow able to update the noise pixels to produce an image that captures the raw style of the original image, with absolutely no structure or meaning.

In [ ]:
Image.open(resultspath+'res_at_iteration_%s.png' % (iterations-1))

### Style transfer

We now know how to reconstruct an image, as well as how to construct an image that captures the style of an original image. The obvious idea may be to just combine these two approaches by weighting and adding the two loss functions.

In [ ]:
# Dimensions of image and style must match
# So we need to do something about it
print (img_shape)
print (style_shape)

Like before, we're going to grab a sequence of layer outputs to compute the style loss. However, we still only need one layer output to compute the content loss. How do we know which layer to grab? As we discussed earlier, the lower the layer, the more exact the content reconstruction will be. In merging content reconstruction with style, we might expect that a looser reconstruction of the content will allow more room for the style to have an effect (re: inspiration). Furthermore, a later layer ensures that the image "looks like" the same subject, even if it doesn't have the same details.

In [ ]:
style_transfer_layers = [style_layer_outputs['block{}_conv1'.format(o)] for o in range(1,6)]
content_transfer_layer_name = 'block4_conv2'
content_transfer_layer = style_layer_outputs[content_transfer_layer_name]
print(style_transfer_layers)
print (content_transfer_layer)

In [ ]:
style_transfer_model = Model(style_model.input, style_layers)
style_transfer_activations = [K.variable(o) 
               for o in style_transfer_model.predict(style_input)]

In [ ]:
print(img_input.shape)
print (style_input.shape)
print (img_model.input)

In [ ]:
content_transfer_model = Model(style_model.input, content_transfer_layer)
content_transfer_activations = K.variable(content_transfer_model.predict(img_input))

Now to actually merge the two approaches is as simple as merging their respective loss functions. Note that as opposed to our previous to functions, this function is producing three separate types of outputs: one for the original image, one for the image whose style we're emulating, and one for the random image whose pixel's we are training. 

One way for us to tune how the reconstructions mix is by changing the factor on the content loss, which we have here as 1/10. If we increase that denominator, the style will have a larger effect on the image, and if it's too large the original content of the image will be obscured by unstructured style. Likewise, if it is too small than the image will not have enough style.

In [ ]:
style_transfer_loss = sum(get_style_loss(l1[0], l2[0]) for l1,l2 in zip(style_transfer_layers, style_transfer_activations))
style_transfer_loss += metrics.mse(content_transfer_layer, content_transfer_activations)/10.
style_transfer_grads = K.gradients(style_transfer_loss, style_model.input)
style_transfer_fn = K.function([style_model.input], [style_transfer_loss]+style_transfer_grads)

In [ ]:
evaluator = Evaluator(style_transfer_fn, output_img_shape)

In [ ]:
iterations=DEFAULT_ITERATIONS
rand_noise_img = gen_rand_noise_img(output_img_shape)

In [ ]:
x = solve_image(evaluator, iterations, rand_noise_img, output_img_shape)

These results are remarkable. Each does a fantastic job at recreating the original image in the style of the artist.

In [ ]:
Image.open(resultspath+'res_at_iteration_%s.png' % (iterations-1))

There are lots of interesting additional things you could try, such as the ideas shown here: https://github.com/titu1994/Neural-Style-Transfer .

In [ ]:
print(x)

In [ ]:
#x = solve_image(evaluator, iterations, x, output_img_shape)

In [ ]:
#Image.open(resultspath+'res_at_iteration_%s.png' % (iterations-1))